## Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Query

#### Public Downloadable Data Sources:
- California DMV Vehicles by Zip Code and Fuel Type
- SDGE Territory Zip Codes (2023)

In [2]:
sdge_zips_data = pd.read_excel("data/SDGE Service Territory Zip Code List Q2-2021.xlsx")

In [3]:
dmv_2019 = pd.read_csv("data/vehicle-fuel-type-count-by-zip-code-2019.csv", low_memory=False)
dmv_2020 = pd.read_csv("data/vehicle-fuel-type-count-by-zip-code-2020.csv", low_memory=False)
dmv_2021 = pd.read_csv("data/vehicle-fuel-type-count-by-zip-code-2021.csv", low_memory=False)
dmv_2022 = pd.read_csv("data/vehicle-fuel-type-count-by-zip-code-2022.csv", low_memory=False)
dmv_2023 = pd.read_csv("data/vehicle-fuel-type-count-by-zip-code-2023.csv", low_memory=False)
dmv_2024 = pd.read_csv("data/vehicle-fuel-type-count-by-zip-code-2024.csv", low_memory=False)

## Data Pre-Processing

In [4]:
sdge_zips = list(sdge_zips_data['ZIP_CODE'])

In [5]:
dmv_2019['Year'] = 2019
dmv_2020['Year'] = 2020
dmv_2021['Year'] = 2021
dmv_2022['Year'] = 2022
dmv_2023['Year'] = 2023
dmv_2024['Year'] = 2024
dmv_2024.rename(columns={'ZIP Code':'Zip Code'}, inplace=True)

In [6]:
dmv_data = pd.concat([dmv_2019, dmv_2020, dmv_2021, dmv_2022, dmv_2023, dmv_2024], axis=0)
dmv_data.drop(columns=['Date'])
dmv_data.rename(columns={'Year':'Registration Year'}, inplace=True)
dmv_data = dmv_data.reset_index(drop=True)
# dmv_data

In [7]:
evs = dmv_data[dmv_data['Fuel'] == "Battery Electric"].reset_index(drop=True)
evs = evs.drop(evs[(evs['Zip Code']=="OOS") | (evs['Zip Code']=="Other")].index)
evs['Zip Code'] = evs['Zip Code'].astype(int)
evs = evs[evs['Zip Code'].isin(sdge_zips)]
evs = evs.groupby(['Registration Year', 'Zip Code'])[['Vehicles']].sum()
evs = evs.reset_index()
evs.head()

## Distribution Analysis

In [8]:
evs.groupby('Registration Year')['Vehicles'].sum()

In [9]:
# Boxplot of EVs by Zip Code by Registration Year
plt.figure(figsize=(12, 6))
sns.boxplot(data=evs, x='Registration Year', y='Vehicles')
plt.title('Distribution of EV Vehicles by Registration Year')
plt.xlabel('Registration Year')
plt.ylabel('Number of Vehicles')
plt.show()

In [10]:
# Violin plot for the distribution of EV vehicles
plt.figure(figsize=(12, 6))
sns.violinplot(data=evs, x='Registration Year', y='Vehicles')
plt.title('Violin Plot of EV Vehicles by Registration Year')
plt.xlabel('Registration Year')
plt.ylabel('Number of Vehicles')
plt.show()

In [11]:
year = 2024
yearly_data = evs[evs['Registration Year'] == year]

# Extract vehicle counts by Zip Code
zip_vehicle_counts = yearly_data.set_index('Zip Code')['Vehicles'].values

# Monte Carlo simulation using poisson-distributed samples
n_simulations = 1000
simulated_totals = []
for _ in range(n_simulations):
    simulated_counts = np.random.poisson(lam=zip_vehicle_counts)  # Generate samples for each zip code
    total_simulated = simulated_counts.sum()  # Sum across zip codes for total count
    simulated_totals.append(total_simulated)


# Plot histogram
observed_total = yearly_data['Vehicles'].sum()  # Total vehicles observed in the year
plt.figure(figsize=(12, 6))
plt.hist(simulated_totals, bins=30, edgecolor='black')
plt.axvline(x=observed_total, color='red', linestyle='--', label=f'Observed Total ({observed_total})')
plt.title(f'Monte Carlo Simulation of Total EV Counts for {year}')
plt.xlabel('Simulated Total EV Counts')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Print summary statistics
simulated_mean = np.mean(simulated_totals)
simulated_std = np.std(simulated_totals)

print("Observed Total:", observed_total)
print("Simulated Mean:", simulated_mean)
print("Simulated Standard Deviation:", simulated_std)

In [12]:
# Group the vehicle counts by Registration Year
yearly_data = evs.reset_index().groupby('Registration Year')['Vehicles'].sum()
observed_counts = yearly_data.values
observed_average = np.mean(observed_counts)

# Monte Carlo simulation using poisson-distributed samples
n_simulations = 1000
simulated_averages = []
for _ in range(n_simulations):
    simulated_counts = np.random.poisson(lam=observed_counts)
    simulated_average = np.mean(simulated_counts)
    simulated_averages.append(simulated_average)

# Plot histogram of simulated means
plt.figure(figsize=(12, 6))
plt.hist(simulated_averages, bins=30, edgecolor='black')
plt.axvline(x=observed_average, color='red', linestyle='--', label=f'Observed Average ({observed_average:.2f})')
plt.title('Monte Carlo Simulation of Average EV Counts Across Years')
plt.xlabel('Simulated Average EV Counts')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Print summary statistics
simulated_mean = np.mean(simulated_averages)
simulated_std = np.std(simulated_averages)

print("Observed Average:", observed_average)
print("Simulated Mean:", simulated_mean)
print("Simulated Standard Deviation:", simulated_std)

In [13]:
# Group the vehicle counts by Registration Year
yearly_data = evs.reset_index().groupby('Registration Year')['Vehicles'].sum()
years = yearly_data.index
observed_counts = yearly_data.values

# Monte Carlo simulation
n_simulations = 1000
simulated_counts = np.random.poisson(lam=observed_counts, size=(n_simulations, len(years)))

# Compute simulated minimums, maximums
simulated_minimums = simulated_counts.min(axis=0)
simulated_maximums = simulated_counts.max(axis=0)

plt.figure(figsize=(12, 6))
# Plot minimum, maximum, and observed values
plt.plot(years, observed_counts, color='red', label='Observed Counts', linewidth=2)
plt.plot(years, simulated_minimums, color='purple', label='Simulated Minimums', linestyle='--', linewidth=2)
plt.plot(years, simulated_maximums, color='blue', label='Simulated Maximums', linestyle='--', linewidth=2)
plt.yscale('log') # Log scale
plt.title('Observed EV Count by Year vs Simulated Minimums and Maximums')
plt.xlabel('Year')
plt.ylabel('Vehicle Count (Log Scale)')
plt.legend()
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.show()